In [1]:
import numpy as np
import util
import pickle
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
import matplotlib.pyplot as plt
import scipy.stats

/Users/Matthew/anaconda2/lib/python2.7/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [2]:
seed=229
state = np.random.RandomState(seed)

trainfile = 'data/train.csv'
header, ids, X, Y = util.fetch_data(trainfile)

test_size = 0.20
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=test_size, random_state=state)

# Training: Randomized Search
Use 10% of the data to conduct a search over XGBoost parameters, then test 

In [ ]:
best_xgb = None

try:
    best_xgb = pickle.load(open('models/xgb_randomized.pickle'))
except IOError:
    pass 

In [ ]:
m_search = X_train.shape[0]/10
X_paramsearch = X_train[:m_search,:]; Y_paramsearch = y_train[:m_search]

param_distros = {'learning_rate': scipy.stats.uniform(loc=0.1,scale=0.1),
                 'min_child_weight': scipy.stats.uniform(loc=0.1,scale=0.9),
                'max_depth': scipy.stats.binom(n=10, p=0.6), 
                 #'gamma': scipy.stats.uniform(loc=0, scale=1),
                 'subsample': scipy.stats.uniform(loc=0.5,scale=0.5),
                 'colsample_bytree': scipy.stats.uniform(loc=0.5, scale=0.5),
                }
xgb = XGBClassifier(silent=0)
cv_random = RandomizedSearchCV(xgb, n_iter=500, param_distributions=param_distros,
                               scoring=util.gini_proba_scorer, verbose=2, n_jobs=-1)
cv_random.fit(X_paramsearch, Y_paramsearch)

best_xgb_score = (-float('inf') if best_xgb == None 
                  else util.gini_proba_scorer(best_xgb, X_paramsearch, Y_paramsearch)) 
                  #.gini_normalized(Y_paramsearch, best_xgb.predict_proba(X_paramsearch)[1:]))

# Update best estimator
if cv_random.best_score_ > best_xgb_score: 
    best_xgb = cv_random.best_estimator_

xgb_out = open('models/xgb_randomized.pickle', 'wb')
pickle.dump(best_xgb, xgb_out)
xgb_out.close()

Fitting 3 folds for each of 500 candidates, totalling 1500 fits
[CV] subsample=0.889730870102, learning_rate=0.173855720703, colsample_bytree=0.54851249975, max_depth=5, min_child_weight=0.322788105024 
[CV] subsample=0.889730870102, learning_rate=0.173855720703, colsample_bytree=0.54851249975, max_depth=5, min_child_weight=0.322788105024 
[CV] subsample=0.889730870102, learning_rate=0.173855720703, colsample_bytree=0.54851249975, max_depth=5, min_child_weight=0.322788105024 
[CV] subsample=0.982999550166, learning_rate=0.137186577101, colsample_bytree=0.735231720695, max_depth=5, min_child_weight=0.418744731918 
[CV]  subsample=0.889730870102, learning_rate=0.173855720703, colsample_bytree=0.54851249975, max_depth=5, min_child_weight=0.322788105024, total=  12.5s
[CV] subsample=0.982999550166, learning_rate=0.137186577101, colsample_bytree=0.735231720695, max_depth=5, min_child_weight=0.418744731918 
[CV]  subsample=0.889730870102, learning_rate=0.173855720703, colsample_bytree=0.5485

[CV]  subsample=0.561122243029, learning_rate=0.143046835273, colsample_bytree=0.52923777809, max_depth=6, min_child_weight=0.973872991351, total=  11.3s
[CV] subsample=0.533261059725, learning_rate=0.17718667901, colsample_bytree=0.630179376688, max_depth=3, min_child_weight=0.875647373597 
[CV]  subsample=0.777015072759, learning_rate=0.191522506061, colsample_bytree=0.658817213399, max_depth=7, min_child_weight=0.724202968472, total=  14.5s
[CV] subsample=0.533261059725, learning_rate=0.17718667901, colsample_bytree=0.630179376688, max_depth=3, min_child_weight=0.875647373597 
[CV]  subsample=0.561122243029, learning_rate=0.143046835273, colsample_bytree=0.52923777809, max_depth=6, min_child_weight=0.973872991351, total=   9.7s
[CV] subsample=0.533261059725, learning_rate=0.17718667901, colsample_bytree=0.630179376688, max_depth=3, min_child_weight=0.875647373597 
[CV]  subsample=0.561122243029, learning_rate=0.143046835273, colsample_bytree=0.52923777809, max_depth=6, min_child_wei

[Parallel(n_jobs=-1)]: Done  33 tasks      | elapsed:  2.3min


[CV] subsample=0.809532949307, learning_rate=0.13793884758, colsample_bytree=0.5909635491, max_depth=6, min_child_weight=0.828339309912 
[CV]  subsample=0.801945281925, learning_rate=0.154408375081, colsample_bytree=0.564876245659, max_depth=7, min_child_weight=0.317725550604, total=  13.3s
[CV] subsample=0.809532949307, learning_rate=0.13793884758, colsample_bytree=0.5909635491, max_depth=6, min_child_weight=0.828339309912 
[CV]  subsample=0.801945281925, learning_rate=0.154408375081, colsample_bytree=0.564876245659, max_depth=7, min_child_weight=0.317725550604, total=  12.7s
[CV] subsample=0.809532949307, learning_rate=0.13793884758, colsample_bytree=0.5909635491, max_depth=6, min_child_weight=0.828339309912 
[CV]  subsample=0.801945281925, learning_rate=0.154408375081, colsample_bytree=0.564876245659, max_depth=7, min_child_weight=0.317725550604, total=  12.9s
[CV] subsample=0.956398038061, learning_rate=0.160424324996, colsample_bytree=0.597792021882, max_depth=8, min_child_weight=

[CV] subsample=0.986374147801, learning_rate=0.138610669489, colsample_bytree=0.720476399962, max_depth=7, min_child_weight=0.596487831092 
[CV]  subsample=0.82926808507, learning_rate=0.116508635242, colsample_bytree=0.687990741379, max_depth=2, min_child_weight=0.916433283003, total=   4.6s
[CV] subsample=0.986374147801, learning_rate=0.138610669489, colsample_bytree=0.720476399962, max_depth=7, min_child_weight=0.596487831092 
[CV]  subsample=0.664026972365, learning_rate=0.195744262214, colsample_bytree=0.653144811908, max_depth=8, min_child_weight=0.563616119745, total=  17.7s
[CV] subsample=0.640801766241, learning_rate=0.140289752909, colsample_bytree=0.602312306636, max_depth=3, min_child_weight=0.982119376533 
[CV]  subsample=0.640801766241, learning_rate=0.140289752909, colsample_bytree=0.602312306636, max_depth=3, min_child_weight=0.982119376533, total=   5.9s
[CV] subsample=0.640801766241, learning_rate=0.140289752909, colsample_bytree=0.602312306636, max_depth=3, min_child

...

[CV]  subsample=0.81023706927, learning_rate=0.116907113645, colsample_bytree=0.65969010322, max_depth=8, min_child_weight=0.254642537521, total=  16.1s
[CV] subsample=0.567608148907, learning_rate=0.161656179318, colsample_bytree=0.546000192267, max_depth=6, min_child_weight=0.408174444236 
[CV]  subsample=0.81023706927, learning_rate=0.116907113645, colsample_bytree=0.65969010322, max_depth=8, min_child_weight=0.254642537521, total=  16.0s
[CV] subsample=0.567608148907, learning_rate=0.161656179318, colsample_bytree=0.546000192267, max_depth=6, min_child_weight=0.408174444236 
[CV]  subsample=0.644647192413, learning_rate=0.188123212323, colsample_bytree=0.989804219865, max_depth=4, min_child_weight=0.978437028167, total=  12.8s
[CV] subsample=0.567608148907, learning_rate=0.161656179318, colsample_bytree=0.546000192267, max_depth=6, min_child_weight=0.408174444236 
[CV]  subsample=0.644647192413, learning_rate=0.188123212323, colsample_bytree=0.989804219865, max_depth=4, min_child_w

[CV]  subsample=0.555968178778, learning_rate=0.188046486492, colsample_bytree=0.791268078253, max_depth=7, min_child_weight=0.44482732439, total=  16.6s
[CV] subsample=0.551003795509, learning_rate=0.125676129392, colsample_bytree=0.846606584217, max_depth=8, min_child_weight=0.379508420307 
[CV]  subsample=0.555968178778, learning_rate=0.188046486492, colsample_bytree=0.791268078253, max_depth=7, min_child_weight=0.44482732439, total=  16.8s
[CV] subsample=0.551003795509, learning_rate=0.125676129392, colsample_bytree=0.846606584217, max_depth=8, min_child_weight=0.379508420307 
[CV]  subsample=0.555968178778, learning_rate=0.188046486492, colsample_bytree=0.791268078253, max_depth=7, min_child_weight=0.44482732439, total=  16.5s
[CV] subsample=0.918062111533, learning_rate=0.139454098969, colsample_bytree=0.599245323161, max_depth=5, min_child_weight=0.471394099356 
[CV]  subsample=0.551003795509, learning_rate=0.125676129392, colsample_bytree=0.846606584217, max_depth=8, min_child_

[CV]  subsample=0.70815909798, learning_rate=0.102447797496, colsample_bytree=0.807132291559, max_depth=5, min_child_weight=0.427720354343, total=  11.2s
[CV] subsample=0.580053930356, learning_rate=0.140350199917, colsample_bytree=0.73808574558, max_depth=5, min_child_weight=0.855479765469 
[CV]  subsample=0.580053930356, learning_rate=0.140350199917, colsample_bytree=0.73808574558, max_depth=5, min_child_weight=0.855479765469, total=  10.7s
[CV] subsample=0.987177441134, learning_rate=0.135212527568, colsample_bytree=0.927540374023, max_depth=7, min_child_weight=0.552248768709 
[CV]  subsample=0.580053930356, learning_rate=0.140350199917, colsample_bytree=0.73808574558, max_depth=5, min_child_weight=0.855479765469, total=  10.8s
[CV] subsample=0.987177441134, learning_rate=0.135212527568, colsample_bytree=0.927540374023, max_depth=7, min_child_weight=0.552248768709 
[CV]  subsample=0.70815909798, learning_rate=0.102447797496, colsample_bytree=0.807132291559, max_depth=5, min_child_we

[CV]  subsample=0.791150232264, learning_rate=0.119514706051, colsample_bytree=0.74308763289, max_depth=8, min_child_weight=0.252467311841, total=  16.5s
[CV] subsample=0.722005294341, learning_rate=0.135627359353, colsample_bytree=0.796547411362, max_depth=4, min_child_weight=0.453575572201 
[CV]  subsample=0.791150232264, learning_rate=0.119514706051, colsample_bytree=0.74308763289, max_depth=8, min_child_weight=0.252467311841, total=  16.6s
[CV] subsample=0.722005294341, learning_rate=0.135627359353, colsample_bytree=0.796547411362, max_depth=4, min_child_weight=0.453575572201 
[CV]  subsample=0.579300659597, learning_rate=0.145798323679, colsample_bytree=0.800743695275, max_depth=5, min_child_weight=0.455373844167, total=  11.4s
[CV] subsample=0.722005294341, learning_rate=0.135627359353, colsample_bytree=0.796547411362, max_depth=4, min_child_weight=0.453575572201 
[CV]  subsample=0.579300659597, learning_rate=0.145798323679, colsample_bytree=0.800743695275, max_depth=5, min_child

[CV]  subsample=0.661522340424, learning_rate=0.154572039841, colsample_bytree=0.980279322504, max_depth=5, min_child_weight=0.144872897776, total=  14.5s
[CV] subsample=0.959273123296, learning_rate=0.123090587389, colsample_bytree=0.678380020004, max_depth=7, min_child_weight=0.445245952792 
[CV]  subsample=0.661522340424, learning_rate=0.154572039841, colsample_bytree=0.980279322504, max_depth=5, min_child_weight=0.144872897776, total=  14.8s
[CV] subsample=0.959273123296, learning_rate=0.123090587389, colsample_bytree=0.678380020004, max_depth=7, min_child_weight=0.445245952792 
[CV]  subsample=0.959273123296, learning_rate=0.123090587389, colsample_bytree=0.678380020004, max_depth=7, min_child_weight=0.445245952792, total=  13.8s
[CV] subsample=0.853223797753, learning_rate=0.183622275118, colsample_bytree=0.825572295947, max_depth=7, min_child_weight=0.684183674004 
[CV]  subsample=0.661522340424, learning_rate=0.154572039841, colsample_bytree=0.980279322504, max_depth=5, min_chi

In [ ]:
trainsizes, traingini, testgini = util.learning_curves(best_xgb, X_train, X_test, y_train, y_test)

In [ ]:
print(traingini)
print(testgini)

In [ ]:
plt.figure()
plt.plot(trainsizes, traingini, label='train gini')
plt.plot(trainsizes, testgini, label='test gini')
plt.xlabel('Training set size')
plt.ylabel('Normalized Gini coefficient')
plt.title('XGBoost')
plt.legend()
plt.savefig('figures/learning_curves/xgboost_randomized.eps')

In [ ]:
plt.show()

In [ ]:
xgb_model

In [ ]:
xgb_model.predict_proba(X_train)